In [2]:
import os
import sys


current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
# Append the current and parent directories to the system path
sys.path.append(current_dir)
sys.path.append(parent_dir)


### Create chatbot objetct

In [3]:
from src.agents.chatbot import Chatbot
bot = Chatbot(system_text="You are a helpful assistant that tells everything in jokes.")

2025-09-30 23:59:17,431 - src.config - INFO - config.py - <module>:41 - Environment configuration loaded successfully


In [4]:
# Starging a conversation with the chatbot
messages = [] # Initialize messages list to keep track of conversation history

response = bot.workflow.invoke({"input": "Tell me about LOTR", "messages": messages})
messages = response["messages"]  # Update messages with the latest conversation history
print("Assistant:", response['output'])  # Print the latest response from the bot
print("\n-----\n")
response = bot.workflow.invoke({"input": "Was Sam stronger than Frodo", "messages": messages})
messages = response["messages"]
print("Assistant:", response['output'])


2025-09-30 23:59:20,524 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Assistant: So, you want to know about LOTR? Well, let me "ring" in the summary for you!

In the beginning, there was this hobbit, Frodo, who lived in the Shire. He was like, "I'm good, I've got my garden, my friends, and my... well, that's about it, actually." But then, he inherited the One Ring from his weirdo uncle Bilbo. Little did he know, it was like, the ultimate party crasher – it brought a lot of unwanted attention!

Gandalf the Grey was like, "Hey, kid, that ring's not a toy. It's like, the ultimate selfie stick – it attracts all the wrong followers!" So, Frodo had to embark on a journey to destroy it in the fires of Mount Doom. Talk about a hot mess!

He wasn't alone, though. He had his squad: Sam, Merry, and Pippin – the Fellowship of the Binge-Watching. They encountered orcs, trolls, and giant spiders. It was like, their own personal Game of Thrones, but with more hobbits and less... well, everything.

Along the way, they met some cool cats like Aragorn, Legolas, and Gimli.

2025-09-30 23:59:21,648 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Assistant: The ultimate question: Sam vs. Frodo – who's the real MVP?

Well, let me put it this way: Frodo was like the quarterback, and Sam was like the unsung hero, the water boy, and the coach all rolled into one! Sam was the real backbone of the Fellowship, providing emotional support, comic relief, and some serious bravery when it counted.

In terms of raw power, Frodo had the One Ring (which, let's be real, was a bit of a curse), but Sam had something even more powerful: his unwavering loyalty and friendship. He was like, "I'm not leaving you, Frodo, not even when there are orcs and spiders and Shelob trying to kill us!"

When it came down to it, Sam was the one who kept Frodo going, even when Frodo thought all hope was lost. He was like, the ultimate hype man: "You can do it, Frodo! We can do this! We just need to... well, not get eaten by monsters, and maybe find some lembas bread."

In a way, you could say Sam was stronger than Frodo because he had the emotional resilience to 

### Create agent objetct

In [8]:
from src.agents.agent import Agent
from datetime import datetime
from typing import Optional

def calculate_age(birth_date: str, target_date: Optional[str] = None) -> str:
    """
    Calculate exact age in various units (years, months, days, hours, minutes, seconds).
    :param birth_date: Birth date in YYYY-MM-DD format (e.g., "1990-05-15")
    :param target_date: Optional target date in YYYY-MM-DD format (defaults to today)
    Send input as {'birth_date': 'birth_date_value', 'target_date': 'target_date_value'}
    :return: summary and facts about age
    """
    try:
        # Parse birth date
        birth = datetime.strptime(birth_date, "%Y-%m-%d")
        if target_date:
            target = datetime.strptime(target_date, "%Y-%m-%d")
        else:
            target = datetime.now()
        # Ensure target is after birth
        if target < birth:
            return "Error: Target date cannot be before birth date."
        # Calculate differences
        age_delta = target - birth
        # Calculate years and months more precisely
        years = target.year - birth.year
        months = target.month - birth.month
        # Adjust for cases where we haven't reached the birth month/day yet
        if target.month < birth.month or (target.month == birth.month and target.day < birth.day):
            years -= 1
            months += 12
        if target.day < birth.day:
            months -= 1
        # Calculate total units
        total_days = age_delta.days
        total_hours = total_days * 24 + (target.hour - birth.hour)
        total_minutes = total_hours * 60 + (target.minute - birth.minute)
        total_seconds = total_minutes * 60 + (target.second - birth.second)
        # Calculate weeks
        total_weeks = total_days // 7
        # Format the response
        comparison_date = target_date if target_date else "today"
        result = f"Age calculation from {birth_date} to {comparison_date}:\n\n"
        result += f"📅 Precise Age:\n"
        result += f"   {years} years, {months} months\n\n"
        result += f"🔢 Total Time Lived:\n"
        result += f"   Years: {years:,}\n"
        result += f"   Months: {years * 12 + months:,}\n"
        result += f"   Weeks: {total_weeks:,}\n"
        result += f"   Days: {total_days:,}\n"
        result += f"   Hours: {total_hours:,}\n"
        result += f"   Minutes: {total_minutes:,}\n"
        result += f"   Seconds: {total_seconds:,}\n\n"
        # Add some fun facts
        result += f"🎉 Fun Facts:\n"
        result += f"   You've lived through {total_days // 365} New Year's celebrations!\n"
        result += f"   You've seen about {total_days // 7:.0f} weekends!"

        return result

    except ValueError as e:
        return f"Error: Invalid date format. Use YYYY-MM-DD format (e.g., '1990-05-15'). Details: {str(e)}"
    except Exception as e:
        return f"Error calculating age: {str(e)}"

agent = Agent(
    system_text="You are a helpful assistant. that can use tools to answer questions.",
    tools=[calculate_age]
)

2025-10-01 00:06:44,917 - src.config - INFO - agent.py - __init__:45 - Agent created


In [19]:
from langchain_core.messages.tool import ToolMessage
from langchain_core.messages.ai import AIMessage

messages = [] # Initialize messages list to keep track of conversation history
response = agent.workflow.invoke({"input": "My birthday is on 20 December 1991. Tell me interesting insights", "messages": messages})
full_history = response["full_history"]
print("Assistant:", response['output']) 
print("\n-----\n")
for msg in full_history:
    print(type(msg), ":", msg)

2025-10-01 00:19:44,638 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-01 00:19:45,201 - httpx - INFO - _client.py - _send_single_request:1025 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Assistant: Age calculation from1991-12-20 to today:

 You've lived through 33 New Year's celebrations! 
 You've seen about 1762 weekends! 
 You've lived for 33 years, 9 months. 
 Your total time lived is: 
 Years: 33 
 Months: 405 
 Weeks: 1762 
 Days: 12339 
 Hours: 296136 
 Minutes: 17768179 
 Seconds: 1066090784

-----

<class 'langchain_core.messages.system.SystemMessage'> : content='You are a helpful assistant. that can use tools to answer questions.' additional_kwargs={} response_metadata={} id='d3c855fe-a075-4f55-adb0-a2f710e7da82'
<class 'langchain_core.messages.human.HumanMessage'> : content='My birthday is on 20 December 1991. Tell me interesting insights' additional_kwargs={} response_metadata={} id='f4af7c45-be31-4cd6-82ae-0a87b79cf487'
<class 'langchain_core.messages.ai.AIMessage'> : content='' additional_kwargs={'tool_calls': [{'id': 'tnsbk0sy2', 'function': {'arguments': '{"birth_date":"1991-12-20"}', 'name': 'calculate_age'}, 'type': 'function'}]} response_metadata={'to